## AI Foundry - Agents v2

For the time being, install Preview version of Foundry SDK to use Agents v2 API:

``` bash
pip install azure-ai-projects --pre
pip install azure-ai-agentserver-agentframework
```

### Environment Setup

In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ImageBasedHostedAgentDefinition, ProtocolVersionRecord, AgentProtocol
from azure.identity import DefaultAzureCredential

In [2]:
# Your Azure AI Foundry project endpoint
PROJECT_ENDPOINT = os.environ.get("AZURE_FOUNDRY_PROJECT_ENDPOINT")
AGENT_NAME = "microsoft-docs-agent-v2"
CONTAINER_IMAGE = "ghcr.io/lazauk/mslearn-mcp-hostedagentv2:latest"
CPU = "1"
MEMORY = "2Gi"

ENVIRONMENT_VARIABLES = {
    "AZURE_FOUNDRY_PROJECT_ENDPOINT": PROJECT_ENDPOINT,
    "AZURE_FOUNDRY_GPT_MODEL": "gpt-4.1-mini",
}

print("✅ Configuration loaded")

✅ Configuration loaded


In [3]:
# Initialize Azure AI Projects client
credential = DefaultAzureCredential()
client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=credential
)

print("✅ Client initialized")

✅ Client initialized


In [4]:
print(f"📦 Creating hosted agent: {AGENT_NAME}")
print(f"   Image: {CONTAINER_IMAGE}")

agent = client.agents.create_version(
    agent_name=AGENT_NAME,
    definition=ImageBasedHostedAgentDefinition(
        container_protocol_versions=[
            ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version="v1")
        ],
        cpu=CPU,
        memory=MEMORY,
        image=CONTAINER_IMAGE,
        environment_variables=ENVIRONMENT_VARIABLES
    )
)

print(f"\n✅ Agent created successfully!")
print(f"   Agent ID: {agent.id}")
print(f"   Agent Name: {agent.name}")
print(f"   Version: {agent.version}")

📦 Creating hosted agent: microsoft-docs-agent-v2
   Image: ghcr.io/lazauk/mslearn-mcp-hostedagentv2:latest

✅ Agent created successfully!
   Agent ID: microsoft-docs-agent-v2:1
   Agent Name: microsoft-docs-agent-v2
   Version: 1


In [ ]:
print("="*70)
print("⚠️ SDK doesn't have a start method. Use ONE of these:")
print("="*70)

print("\nOption 1 - Azure Portal:")
print("   1. Go to: https://ai.azure.com")
print(f"   2. Open project: ai-projectv2-swc")
print("   3. Click on Agents")
print(f"   4. Find: {AGENT_NAME}")
print("   5. Click 'Start agent deployment' button")

print("\nOption 2 - Azure CLI:")
print(f"az cognitiveservices agent start \\")
print(f"  --account-name ai-projectv2-swc-resource \\")
print(f"  --project-name ai-projectv2-swc \\")
print(f"  --name {AGENT_NAME} \\")
print(f"  --agent-version {agent.version}")

In [ ]:
print("🧪 Testing deployed agent...")

# Get OpenAI client
openai_client = client.get_openai_client()

# Create conversation
conversation = openai_client.conversations.create()
print(f"✅ Conversation: {conversation.id}")

# Send test message
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body={
        "agent": {
            "name": AGENT_NAME,
            "type": "agent_reference"
        }
    },
    input="How to create an Azure Function in Python?",
)

print("\n📝 Agent Response:")
print("="*70)
print(response.output_text)
print("="*70)

### Housekeeping

In [ ]:
print("📋 All agents in your project:")
agents = client.agents.list()
for agent in agents:
    print(f"\n   Agent: {agent.name}")
    versions = client.agents.list_versions(agent_name=agent.name)
    for v in versions:
        print(f"      - Version {v.version}")

In [ ]:
print("🗑️ Deleting agent...")

try:
    # Delete specific version
    client.agents.delete_version(
        agent_name=AGENT_NAME,
        agent_version="1"
    )
    print(f"✅ Deleted {AGENT_NAME}:1")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n✅ Cleanup complete")